### CS4423 - Networks
Prof. Götz Pfeiffer<br />
School of Mathematics, Statistics and Applied Mathematics<br />
NUI Galway

# Assignment 5

Provide answers to the problems in the boxes provided.  

The buttons at the top of the page can be used to **create
more boxes if needed**.
The type of box can be changed from `Code` to `Markdown`.
`Code` boxes take (and execute) `python` code.
`Markdown` boxes take (and format nicely) **text input**.
In this way, you can provide answers, ask questions, 
or raise issues, in words.

Some marks will be awarded for
participation and engagement.

When finished, print this notebook into a **pdf** file and submit this to
**blackboard**.

**Deadline** is this **Friday** at 5pm.

## Setup

This is a `jupyter` notebook.   You can open and interact
with the notebook through one of sites recommended at
its [github](https://github.com/gpfeiffer/cs4423) repository.

Or, you can
install and use `jupyter` as a `python` package on your own laptop or PC.  

The following command loads the `networkx` package into the current session.  
The next command specifies some standard options that can be useful for drawing graphs.  

In order to execute the code in a box,
use the mouse or arrow keys to highlight the box and then press SHIFT-RETURN.

In [ ]:
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
opts = { "with_labels": True, "node_color": 'y' }

Should it ever happen that the notebook becomes unusable, start again with a fresh copy.

## 1.  A Citation Network

The file `scientometrics.net` contains a representation of a network of citations between the $1656$ articles that
appeared from 1978 to 2004 in [Scientometrics](https://en.wikipedia.org/wiki/Scientometrics_(journal)), 
a journal devoted to the field of bibliometrics and the "Science of Science".  It is loaded into a graph `G`
as follows.

In [ ]:
E = nx.read_edgelist("data/scientometrics.net", create_using=nx.DiGraph, nodetype=int)
G = nx.DiGraph()
G.add_nodes_from(range(1, 1656))
G.add_edges_from(E.edges())

`G` as **a lot of nodes** and even more edges:  **Don't draw** this network.

In [ ]:
m, n = G.order(), G.size()
m, n

The file `scientometrics_paper_year.txt` contains, for most but not all articles in the network, 
their year of publication.  It is loaded and stored as node attributes as follows

In [ ]:
years = np.loadtxt("data/scientometrics_paper_year.txt", dtype="int")
for date in years:
    G.nodes[date[0]]['year'] = date[1]

Now most nodes have their publication year as value of the `'year'` key in their attributes dictionary.
For example, node $1113$.

In [ ]:
G.nodes[1113]['year']

If the publication year of a node is not known, the normal dictionary access by the key `'year'` will fail
with an error message.  Using the method `get()` instead will return `None` in such a case.

In [ ]:
print(G.nodes[4].get('year'))

In a directed network, one distinguishes between the **in-degree** and the **out-degree** of a node.
This distinction leads to **two** separate degree distributions.   The out-degree distribution
can be computed as follows.

In [ ]:
outs = {}
for (n, d) in G.out_degree():
    outs[d] = outs.get(d, 0) + 1

outs = list(outs.items())
outs.sort()
print(outs)

Excluding the degree $0$ count, a loglog plot of the out-degree frequencies
looks as follows.

In [ ]:
df = pd.DataFrame(outs)
df.iloc[1:].plot.scatter(x = 0, y = 1, loglog=True)

Does that look like a power law degree distribution?

* If you were to draw a straight line through the plot, approximating the data points, what
would the slope of that line be, approximately?
* Following the above example, **compile a histogram of the in-degrees** of the graph `G`,
discard the $0$-degree nodes, and then create a loglog plot of the remaining data points.
* If you were to draw a straight line through the plot, approximating the data points, what
would the slope of that line be, approximately?
* Can you go back in time and recover the state of the network at the end of $1998$ (as a subgraph `G0`
of `G`)?  If so, for articles published from $1999$ onwards, determine the frequency of
articles in `G0` being cited, in relation to their degree in `G0`, and plot
the result in a meaningful way.

(To enable input for the text cell below, highlight the cell and press return.
To typeset the text nicely, and disable input, type SHIFT-return.)

... text goes here ...

### 2. Configuration Model

A configuration model for a given list of stubs (half-edges) can be produced with the following `python` program:

In [ ]:
import random

def configuration_stubs(stubs):
    m = len(stubs) // 2  # should check if sum(degrees) is even ...
    random.shuffle(stubs)
    G = nx.Graph(zip(stubs[:m], stubs[m:]))
    G.remove_edges_from(nx.selfloop_edges(G))
    return G

The following function produces a list of stubs from a list `a` of degrees:

In [ ]:
def stubs_list(a):
    return sum([d * [i] for i, d in enumerate(a)], [])

So we can start with a list `dd` of degrees, convert this into a list of stubs and then
create a graph `G` with the given list of node degrees (up to self-loops and multiple edges).

In [ ]:
dd = [3, 2, 2, 1, 1, 1]
stubs = stubs_list(dd)
G = configuration_stubs(stubs)
print(sum(dd)//2, G.size())

In [ ]:
nx.draw(G, **opts)

Conversely, we can use `collections.Counter` to recover the list of degrees from the list of stubs:

In [ ]:
stubs

In [ ]:
from collections import Counter

In [ ]:
k = Counter(stubs)
print(k)

The following function produces a list of stubs, whose corresponding list of degrees is
distributed according to a power law $p_k = n_k/n \approx c k^{-\gamma}$ with parameter
$\gamma = \frac{2-p}{1-p}$.

In [ ]:
def powerstubs(m, p):
    
    # distribute 2*m according to a power law
    l = 0
    x = [l]
    for i in range(2*m-1):
        if random.random() < p:
            l += 1
            x.append(l)
        else:
            k = random.choice(x)
            x.append(k)
            
    return x

In [ ]:
stubs = powerstubs(10, 1/2)
stubs

In [ ]:
degrees = list(Counter(stubs).values())
degrees

* Construct a list `stubs` and a corresponding list of approximately $n = 5000$ `degrees` such that
  the distribution of the values in `degrees` follows a power law.
* What is the value of $\gamma$ in this power law?
* Use the above function `configuration_stubs` to create
  a **random network** `C` with degree sequence `degree`.
* How many nodes and edges do you expect, how many nodes and edges does the graph
  `C` actually have?
* In `networkx`, a configuration model can be generated with the function 
  `nx.configuration_model(degrees)`.  Apply this function to the same degree
  sequence to obtain a random network `D`.   How many nodes and edges does
  the graph `D` have?
* Can you draw a loglog plot that provides evidence for `D` having a power law
  degree distribution?

(If $n = 5000$ turns out to be to large, use a smaller number of nodes.)

### 3. Preferential Linear Attachment

The following algorithm, introduced and named after Barabási-Albert, **grows** a network
on $n$ nodes from a complete graph on $a$ nodes, by
adding one node at a time, together with $b$ new edges, linking the new node to 
$b$ old nodes, with a probability given by their current degrees:

An $(n, a, b)$**-BA model** is a (simple) graph on $n$ nodes, constructed as follows.
1. start with a complete graph on $a$ nodes (at time $t = 0$)
2. for $t = 1, \dots, n-a-1$:
    * add new node $x = a + t$
    * and $b$ links to old nodes $i$ with probability 
      $$
      p_{x \to i} = \frac{k_{i, t-1}}{2 m_{t-1}},
      $$
      where $k_{i,t}$ is the degree of node $i$ at time $t$ and $m_t$ is the
      number of edges at time $t$.

For an implementation of this algorithm, we need to be able to draw $m$ items from the current
list of nodes, where each node is picked with a probability proportional to its current degree
This functionality is provided by the function `random.choices`,

In [ ]:
import random

To pick $4$ from $0,\dots,5$ with a large bias towards $5$:

In [ ]:
sorted(random.choices(range(6), 5*[1] + [5], k=4))

No we are ready to grow a graph.  Let's set $a = 3$ and $b = 2$ for starters.

In [ ]:
a, b = 3, 2

In [ ]:
G = nx.complete_graph(a)
G.degree()

In [ ]:
x, m = G.order(), G.size()
x, m

Compute a discrete probabilty distribution corresponding to node degrees.

In [ ]:
degrees = [d for n, d in G.degree()]
degrees

Select $b$ distinct nodes from the old graph according to the probabilities `prob`

In [ ]:
old = sorted(random.choices(list(G.nodes()), degrees, k=b))
old

Add edges from a new node `x` to the selected nodes.

In [ ]:
G.add_edges_from([(x, o) for o in old])

Draw the resulting graph.

In [ ]:
nx.draw(G, **opts)

For the next node, all of the previous steps are repeated.

In [ ]:
x, m  = G.order(), G.size()
degrees = [d for n, d in G.degree()]
old = sorted(random.choices(list(G.nodes()), degrees, k=b))
G.add_edges_from([(x, o) for o in old])

And again, we can draw the resulting graph.

In [ ]:
nx.draw(G, **opts)

And so on ...

* Add 20 more nodes to the graph `G` by repeating the above steps
  and draw the resulting graph.
* Add 250 more nodes to the graph `G` by repeating the above steps
  (preferrably with a python `for` loop, rather than typing out 250 identical pieces of code.)
* Compute the graph clustering coefficient $C$ and the characteristic path length $L$ of `G`.
  How do these values compare to ER-random graphs of similar size?
* Create a loglog plot of the degree histogram of the final version of `G`: does this
  indicate a power law distribution?  If so, what is the exponent $\gamma$?
* In `networkx`, a BA-model can be generated with the function
  `nx.barabasi_albert_graph(n, b)` (where $a = b$ in terms of our parameters, and the initial
  graph is an **empty** graph on $a$ nodes).  Use this function to generate a graph `B`
  comparable to `G` above, and check whether its degree distribution follows a power law.

### 4. Course Summary

* In your own words, summarize the contents of the course.  What is Network Science?  Which parts are covered in the course, which are missing?  Use complete sentences.  Write approximately 100 words.

... text goes here ...